In [ ]:
import sympy
import numpy as np
from math import factorial
from sympy.interactive import printing

printing.init_printing(use_unicode=True) 

In [ ]:
sympy.__version__

In [ ]:
from time import time

In [ ]:
wanted_order = 5

In [ ]:
t1 = time()

# Defining symbols
H0_AA, H0_BB, H1_AA, H1_BB, H2_AB, H2_BA, S_AB, S_BA = sympy.symbols('H0_AA, H0_BB, H1_AA, H1_BB, H2_AB, H2_BA, S_AB, S_BA', commutative=False)
x = sympy.Symbol('x', commutative=True)
S_AB_i = sympy.symbols('S_AB_0:9', commutative=False)
S_BA_i = sympy.symbols('S_BA_0:9', commutative=False)

S_AB_series = 0
S_BA_series = 0
for i in range(1, wanted_order):
    S_AB_series += S_AB_i[i] * x ** (i)
    S_BA_series += S_BA_i[i] * x ** (i)

def commute_n_odd(H_AA, H_BB, S_AB, S_BA, n):
    assert n % 2 == 1
    res_AB = H_AA * S_AB - S_AB * H_BB
    res_BA = H_BB * S_BA - S_BA * H_AA
    for i in range((n - 1) // 2):
        res_AA = res_AB * S_BA - S_AB * res_BA
        res_BB = res_BA * S_AB - S_BA * res_AB
        res_AB = res_AA * S_AB - S_AB * res_BB
        res_BA = res_BB * S_BA - S_BA * res_AA
    return res_AB

def commute_n_even(H_AB, H_BA, S_AB, S_BA, n):
    assert n % 2 == 0
    res_AB = H_AB
    res_BA = H_BA
    for i in range(n // 2):
        res_AA = res_AB * S_BA - S_AB * res_BA
        res_BB = res_BA * S_AB - S_BA * res_AB
        res_AB = res_AA * S_AB - S_AB * res_BB
        res_BA = res_BB * S_BA - S_BA * res_AA
    return res_AB


# Calculating offdiagonal 
Hn_AB = 0
for j in range(wanted_order//2 + 1):
    Hn_AB += commute_n_odd(H0_AA + H1_AA, H0_BB + H1_BB, S_AB, S_BA, 2*j+1) / sympy.factorial(2*j+1)
    Hn_AB += commute_n_even(H2_AB, H2_BA, S_AB, S_BA, 2*j)  / sympy.factorial(2*j)

print('step 1')

# Removing not needed stuff
Hn_AB = Hn_AB.subs(S_AB, S_AB*x)
Hn_AB = Hn_AB.subs(S_BA, S_BA*x)
Hn_AB = Hn_AB.subs(H1_AA, H1_AA*x)
Hn_AB = Hn_AB.subs(H1_BB, H1_BB*x)
Hn_AB = Hn_AB.subs(H2_AB, H2_AB*x)
Hn_AB = Hn_AB.subs(H2_BA, H2_BA*x)

print('step 2')

collected = Hn_AB.expand().collect(x, evaluate=False)
Hn_AB = 0
for n in range(1, wanted_order+1):
    Hn_AB += collected[x**n]

print('step 3')    
    
# Getting important stuff
Hn_AB = Hn_AB.subs(H1_AA, H1_AA*x)
Hn_AB = Hn_AB.subs(H1_BB, H1_BB*x)
Hn_AB = Hn_AB.subs(H2_AB, H2_AB*x)
Hn_AB = Hn_AB.subs(H2_BA, H2_BA*x)
Hn_AB = Hn_AB.subs(S_AB, S_AB_series)
Hn_AB = Hn_AB.subs(S_BA, S_BA_series)
collected = Hn_AB.expand().collect(x, evaluate=False)

t2 = time()
print((t2-t1)/60)

In [ ]:
Y_i = []
for n in range(1, wanted_order + 1):
    Y = - (collected[x**n] - (H0_AA*S_AB_i[n] - S_AB_i[n]*H0_BB))
    Y = Y.expand()
    display(Y)
    Y_i.append(Y)

# for computations

In [ ]:
def write_n(n):
    expr = - (collected[x**n] - (H0_AA*S_AB_i[n] - S_AB_i[n]*H0_BB))
    expr.expand()
    
    output = ""
    output += 'def Y%i(H0_AA, H0_BB, H1_AA, H1_BB, H2_AB, H2_BA, S_AB_i, S_BA_i):\n' % n
    if n > 1:
        output += '    '
        for i in range(1,n):
            output += 'S_AB_%i, ' % i
        output += '= S_AB_i[:%i]\n' % (n-1)
        output += '    '
        for i in range(1,n):
            output += 'S_BA_%i, ' % i
        output += '= S_BA_i[:%i]\n' % (n-1)
    output += '    return (' + str(expr) + ')\n'
    return output

In [ ]:
text_to_write = u""

for n in range(1, wanted_order+1):
    text_to_write += write_n(n)
    text_to_write += '\n\n'

text_to_write += 'Y_i = ('
for n in range(1, wanted_order+1):
    text_to_write += 'Y%i,' % n
text_to_write += ')\n'
    
import io

with io.open('perturbationS.py', 'w') as f:
    f.write(text_to_write)    